In [8]:
!pip install PyMuPDF

In [9]:
import pymupdf
import fitz
import pandas as pd

In [10]:
# doc = fitz.open('2023.12.31.PortSum.Vista V..pdf')
doc = pymupdf.open('2023.12.31.PortSum.Vista V..pdf')
page1 = doc[6]
words = page1.get_text("words")

In [11]:
type(page1)

pymupdf.Page

In [12]:
words[10]

(117.36285400390625,
 120.15873718261719,
 123.63448333740234,
 129.14093017578125,
 'in',
 4,
 1,
 1)

In [13]:
tables=[]
for table in page1.find_tables().tables:
    tables.append(table)

In [15]:
#tables[2].to_pandas()

In [16]:
# deployment_name = "gpt-4o"
# openai_api_type = "azure"
# openai_api_key = "992ce21baef84c68a10972ffa78c4eeb"
# openai_api_base = "https://usw1d-openai01.openai.azure.com/"
# openai_api_version = "2024-02-01"

In [17]:
import os
import subprocess
import base64
!export AZURE_API_VERSION="`kubectl get secret azure-creds -n kubeflow-brad -o jsonpath='{.data.AZURE_API_VERSION}' | base64 --decode`"
!export AZURE_OPENAI_API_KEY="`kubectl get secret azure-creds -n kubeflow-brad -o jsonpath='{.data.AZURE_OPENAI_API_KEY}' | base64 --decode`"
!export AZURE_OPENAI_ENDPOINT="`kubectl get secret azure-creds -n kubeflow-brad -o jsonpath='{.data.AZURE_OPENAI_ENDPOINT}' | base64 --decode`"
def kubectl_get_secret(secret_name, key):
    # Executing kubectl command to get the secret and extracting the value of the specified key
    command = f"kubectl get secret {secret_name} -n kubeflow-brad -o jsonpath={{.data.{key}}}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)

    # Checking if the command was successful
    if result.returncode == 0:
        # Decoding the base64-encoded value and return
        decoded_value = base64.b64decode(result.stdout.strip()).decode('utf-8')
        return decoded_value

# Fetching values from Kubernetes secret and decode
AZURE_API_VERSION = kubectl_get_secret("azure-creds", "AZURE_API_VERSION")
AZURE_OPENAI_API_KEY = kubectl_get_secret("azure-creds", "AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = kubectl_get_secret("azure-creds", "AZURE_OPENAI_ENDPOINT")

# Setting the environment variables
os.environ["AZURE_API_VERSION"] = AZURE_API_VERSION
os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT

Error from server (Forbidden): secrets "azure-creds" is forbidden: User "system:serviceaccount:kubeflow-grant-kim:default-editor" cannot get resource "secrets" in API group "" in the namespace "kubeflow-brad"
Error from server (Forbidden): secrets "azure-creds" is forbidden: User "system:serviceaccount:kubeflow-grant-kim:default-editor" cannot get resource "secrets" in API group "" in the namespace "kubeflow-brad"
Error from server (Forbidden): secrets "azure-creds" is forbidden: User "system:serviceaccount:kubeflow-grant-kim:default-editor" cannot get resource "secrets" in API group "" in the namespace "kubeflow-brad"


TypeError: str expected, not NoneType

In [ ]:
AZURE_OPENAI_ENDPOINT

In [ ]:
from langchain_openai import AzureChatOpenAI
llm = AzureChatOpenAI(
    openai_api_version=os.getenv("AZURE_API_VERSION"),
    azure_deployment="gpt-4o",
    temperature=0
)

In [ ]:
from langchain_core.prompts.chat import ChatPromptTemplate
#langchain.debug = False

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Help me with my math homework!"),
        ("human", "{user_input}" ),
    ]
)
messages = chat_template.format_messages(
  user_input="Hello! Could you solve 2+2?"
)
ai_message = llm.invoke(messages)
print(ai_message.content)

# response = openai.chat.completions.create(
#   model="gpt-4o",
#   messages=[
#     {
#       "role": "user",
#       "content": [
#         {"type": "text", "text": "What’s in this image?"},
#         {
#           "type": "image_url",
#           "image_url": {
#             "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
#           },
#         },
#       ],
#     }
#   ],
#   max_tokens=300,
# )

In [ ]:
from IPython.display import Image, display, Audio, Markdown
import base64

# IMAGE_PATH = "shared/projects/sample-notebooks/quarterly_reports_project/Grant Kim/stepstone1.png"
IMAGE_PATH = "page15_image_4.jpg"

# Preview image for context
display(Image(IMAGE_PATH))

# Open the image file and encode it as a base64 string
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

base64_image = encode_image(IMAGE_PATH)

messages=[
        {"role": "system", "content": "You are a Data Analyst."},
        {"role": "user", "content": [
            # {"type": "text", "text": "Given the following image, if it is a table, please return a tuple, where the first element is 'table' and the second element is the table in html table format, starting and ending with the <table> tags.  If it is a Brand Logo, please return a tuple where the first element is 'title' and the second element is any text that can be extracted from the logo (if there is no text, return whitespace.  If it is neither, return an empty tuple"},
            {"type": "text", "text": "Given the following image, if it is a table, please return a tuple, where the first element is 'table' and the second element is a paragraph capturing all of the information in the table, in sentence form.  If it is a Brand Logo, please return a tuple where the first element is 'title' and the second element is any text that can be extracted from the logo (if there is no text, return whitespace.  If it is neither, return an empty tuple"},
            {"type": "image_url", "image_url": {
                "url": f"data:image/png;base64,{base64_image}"}
            }
        ]}
    ]
ai_message = llm.invoke(messages)
print(ai_message.content)

In [ ]:
!pip install ultralyticsplus ultralytics
#!pip uninstall -y fitz
#!pip install --upgrade --force-reinstall pymupdf
#!pip install frontend
#!pip install tools
#!pip install -U opencv-python-headless
#!pip install opencv-python
#!pip install sahi
#!pip install -U ultralytics

In [ ]:
#cat /opt/conda/lib/python3.11/site-packages/sahi/utils/cv.py 

In [ ]:
#import fitz
import pymupdf as fitz
import pymupdf
import sys
import os
import glob

# pdf_path: the folder of all the PDF files
# saved_path: the path of the saved page images
def convert_pdf_to_image(pdf_path, pdf_file, saved_path):
    
    if not os.path.exists(saved_path):
        os.mkdir(saved_path)
    else:
        files = glob.glob(saved_path+'/*')
        print(files)
        for f in files:
            os.remove(f)

    try:
        print("got here 1")
        print(pdf_path + "/" + pdf_file)
        fitz.TOOLS.mupdf_warnings()  # empty the problem message container
        doc = fitz.open(pdf_path + "/" + pdf_file)
        print("one: "+doc)
        warnings = fitz.TOOLS.mupdf_warnings()
        if warnings:
            print(warnings)
            raise RuntimeError()

        for page in doc:  # iterate through the pages
            pix = page.get_pixmap()  # render page to an image
            pix.save(saved_path + "/" + f"{pdf_file[:-4]}-{page.number}.png")  # store image as a PNG
        return 

    except:
        print("error when opening the pdf file {}".format(pdf_file))
        return None

In [ ]:
for page in doc:  # iterate through the pages
    pix = page.get_pixmap()  # render page to an image
    pix.save("/home/jovyan/shared/projects/sample-notebooks/quarterly_reports_project/Grant Kim/static/images/" + f"{'FloridaSample.pdf'[:-4]}-{page.number}.png")  # store image as a PNG


In [ ]:
notebook_path = "/home/jovyan/shared/projects/sample-notebooks/quarterly_reports_project/Grant Kim/"
data_path = notebook_path+"static/pdfs"
converted_path = notebook_path+"static/images"
dir = os.listdir(data_path)
print(dir)
for file in dir:
   pdf_name = file
   convert_pdf_to_image(data_path, pdf_name, converted_path)

In [ ]:
dir = os.listdir(data_path)
print(dir)
for file in dir:
   pdf_name = file
   convert_pdf_to_image(data_path, pdf_name, converted_path)



In [ ]:
from ultralyticsplus import YOLO, render_result

# load model
model = YOLO('keremberke/yolov8m-table-extraction')

# set model parameters
model.overrides['conf'] = 0.25  # NMS confidence threshold
model.overrides['iou'] = 0.45  # NMS IoU threshold
model.overrides['agnostic_nms'] = False  # NMS class-agnostic
model.overrides['max_det'] = 10  # maximum number of detections per image

# set image
image = '/home/jovyan/shared/projects/sample-notebooks/quarterly_reports_project/Grant Kim/static/images/FloridaSample-1.png'

# perform inference
results = model.predict(image)

# observe results
print(results[0].boxes)
render = render_result(model=model, image=image, result=results[0])
render.show()


In [ ]:
#!apt-get install -y libgl1-mesa-glx

In [ ]:
import cv2

image = cv2.imread('/home/jovyan/shared/projects/sample-notebooks/quarterly_reports_project/Grant Kim/static/images/FloridaSample-1.png')
copy = image.copy()

# ROI = image[y:y+h, x:x+w]
# xywh: tensor([[395.3298, 377.0665, 645.2010, 376.8942],
#         [399.3792,  94.9245, 649.2988,  55.3959]], device='cuda:0')
# xyxy: tensor([[ 72.7293, 188.6194, 717.9304, 565.5136],
#         [ 74.7298,  67.2265, 724.0286, 122.6224]], device='cuda:0')

# ROI = image[377:377+377, 395:395+646]
ROI = image[189:566, 72:718]
#ROI = image[67:122,74:724]
cv2.imwrite(converted_path+'/cut.png', ROI)
# cv2.rectangle(copy,(x,y),(x+w,y+h),(36,255,12),2)

# cv2.imshow('thresh', thresh)
# cv2.imshow('copy', copy)
cv2.waitKey()